# 405 DQN Reinforcement Learning V2

In [1]:
import torch
import torch.nn as nn 
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

In [2]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01
EPSILON = 0.9
GAMMA = 0.9
TARGET_REPLACE_ITER = 100
MEMORY_CAPACITY = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
N_ACTIONS = env.action_space.n 
N_STATES = env.observation_space.shape[0]

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [4]:
net = Net()
actions_value = net.forward(Variable(torch.randn(8, N_STATES)))

In [5]:
actions_value

tensor([[0.1705, 0.0082],
        [0.1895, 0.0391],
        [0.2014, 0.0339],
        [0.2134, 0.0029],
        [0.1751, 0.0395],
        [0.1325, 0.0062],
        [0.1999, 0.0111],
        [0.1340, 0.0089]], grad_fn=<AddmmBackward0>)

In [6]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        self.learn_step_counter = 0
        self.memory_counter = 0
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
        
    def choose_action(self, x):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        if np.random.uniform() < EPSILON:
            actions_value = self.eval_net.forward(x)
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0]
            ## TODO: WHY
        else:
            action = np.random.randint(0, N_ACTIONS)
        return action
    
    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
        
    def learn(self):
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            print("Replace Learning: ", self.learn_step_counter)
            self.target_net.load_state_dict(self.eval_net.state_dict()) ## TODO: WHY
        self.learn_step_counter += 1
        
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = Variable(torch.FloatTensor(b_memory[:, :N_STATES]))
        b_a = Variable(torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int)))
        b_r = Variable(torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2]))
        b_s_ = Variable(torch.FloatTensor(b_memory[:, -N_STATES:]))
        
        q_eval = self.eval_net(b_s).gather(1, b_a)
        q_next = self.target_net(b_s_).detach()
        q_target = b_r + GAMMA * q_next.max(1)[0]
        loss = self.loss_func(q_eval, q_target)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
            

In [7]:
dqn = DQN()

In [8]:
print("\nCollecting experience...")
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        env.render()
        a = dqn.choose_action(s)
        
        s_, r, done, info = env.step(a)
        
        # modify the reward
        x, x_dot, theta, theta_dot = s_
        r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        r = r1 + r2
        
        dqn.store_transition(s, a, r, s_)
        
        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))
        if done:
            break 
        s = s_

c:\Users\29468\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\29468\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Ep:  202 | Ep_r:  2.35
Ep:  203 | Ep_r:  2.62
Ep:  204 | Ep_r:  3.11
Ep:  205 | Ep_r:  1.51
Ep:  206 | Ep_r:  2.59
Ep:  207 | Ep_r:  2.07
Ep:  208 | Ep_r:  1.18
Ep:  209 | Ep_r:  1.86
Ep:  210 | Ep_r:  3.82
Ep:  211 | Ep_r:  2.91
Ep:  212 | Ep_r:  3.92
Ep:  213 | Ep_r:  2.85
Ep:  214 | Ep_r:  1.21
Ep:  215 | Ep_r:  1.87
Ep:  216 | Ep_r:  2.82
Ep:  217 | Ep_r:  3.04
Ep:  218 | Ep_r:  2.86
Ep:  219 | Ep_r:  1.21
Ep:  220 | Ep_r:  2.37
Ep:  221 | Ep_r:  2.87
Ep:  222 | Ep_r:  2.46
Ep:  223 | Ep_r:  3.59
Ep:  224 | Ep_r:  2.5
Ep:  225 | Ep_r:  2.37
Ep:  226 | Ep_r:  2.16
Ep:  227 | Ep_r:  1.25
Ep:  228 | Ep_r:  1.23
Ep:  229 | Ep_r:  1.93
Ep:  230 | Ep_r:  4.69
Ep:  231 | Ep_r:  2.95
Ep:  232 | Ep_r:  0.96
Ep:  233 | Ep_r:  1.06
Ep:  234 | Ep_r:  2.51
Ep:  235 | Ep_r:  3.14
Ep:  236 | Ep_r:  4.22
Ep:  237 | Ep_r:  3.03
Ep:  238 | Ep_r:  3.13
Ep:  239 | Ep_r:  2.33
Ep:  240 | Ep_r:  3.1
Ep:  241 | Ep_r:  3.09
Ep:  242 | Ep_r:  1.22
Ep:  243 | Ep_r:  3.07
Ep:  244 | Ep_r:  2.4
Ep:  245 | Ep_